# 00 settings

In [2]:
import scipy.io as scio
import h5py
import numpy as np
import tifffile as tf
from PIL import Image
import hdf5storage

import pandas as pd
import numpy as np
import math
import random
import copy
from itertools import chain

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.gridspec import GridSpec
import seaborn as sns
import networkx as nx
from networkx.algorithms import community
import powerlaw

from scipy.stats import pearsonr
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.signal import find_peaks, correlate2d
from scipy import stats
from scipy import signal

ele_start_second = 239 ## change it according to your ecog csv file!
calc_start, calc_end = 2400, 3800  ## change it according to your anesthesia period

res_path = 'xxx/data_analysis_res_m010/'
load_path = 'xxx/data_m010/'

print("res_path:", res_path)
print("load_path:", load_path)

res_path: xxx/data_analysis_res_m010/
load_path: xxx/data_m010/


# 01 load data

In [ ]:
'''
calc data
'''
denoised_data = np.load(res_path + "records/neuron_denoised_records_whole_brain.npy")
print("denoised_data:", denoised_data.shape)

fig,ax = plt.subplots(figsize=(7,4), dpi=100,facecolor="w")
caxes = ax.matshow(denoised_data, vmin = -1.5, vmax = 1.5, cmap = "viridis", aspect = "auto")
fig.colorbar(caxes)
plt.title("whole brain denoised_data")
plt.show()

In [ ]:
'''
ecog data:  Change it according to your csv file!
'''
name = "burst_supp"

ele_rec_bs = pd.read_csv(load_path + "ele_trigger/ele_signal_" + name + "_fp01-fp32.csv")
ele_rec_bs = ele_rec_bs.values
ele_rec_bs = ele_rec_bs.T
print("ele_rec_bs:", type(ele_rec_bs), ele_rec_bs.shape)

ele_fp26 = ele_rec_bs[26 * 2 - 1]
print("ele_fp26:", len(ele_fp26))
plt.plot(ele_fp26)
plt.show()


# 02 plot traces

In [ ]:
'''
burst neurons
'''

name = "burst"
ele_start, ele_end = calc_start / 10 + 10.4, calc_end / 10 + 10.4
ele_start_row, ele_end_row = int((ele_start - ele_start_second) * 1000), int((ele_end - ele_start_second) * 1000)
print("ele_start_row, ele_end_row:", ele_start_row, ele_end_row)

# plot ecog data
ele_rec = ele_fp26[ele_start_row : ele_end_row]
fig,ax = plt.subplots(figsize=(11,4), dpi=100, facecolor="w")
plt.plot(ele_rec, color = "#17202A", linewidth = 0.4, alpha = 0.8)
plt.title("ECoG signal", fontsize = 18)
plt.ylabel("(mV)", fontsize = 14)
plt.xlabel("Time", fontsize = 14)
plt.xticks(np.arange(0, ele_end_row - ele_start_row , 10000), np.arange(calc_start, calc_end , 100))
plt.show()

# load neuron ids
select_data_ids = np.load(res_path + "neurons/neurons_ids_" + name + "_all_start.npy")  ## change the file name
print("select_data_ids:", len(select_data_ids))


# plot calcium data 1
fig,ax = plt.subplots(figsize=(11,4), dpi=100, facecolor="w")
for i in select_data_ids:
    plt.plot(denoised_data[i, calc_start : calc_end], alpha = 0.8, color = "#2471A3", linewidth = 0.8)
plt.title("burst neurons", fontsize = 18)
plt.xticks(np.arange(0, calc_end - calc_start, 100), np.arange(calc_start, calc_end, 100))
plt.ylabel("\u0394 F / F", fontsize = 14)
plt.xlabel("Time", fontsize = 14)
plt.savefig(res_path + "neurons/calc_records_" + name + "_select_data_line.png", bbox_inches='tight')
plt.savefig(res_path + "neurons/calc_records_" + name + "_select_data_line.pdf", bbox_inches='tight')
plt.show()


# plot calcium data 2
calc_rec = denoised_data[select_data_ids, calc_start : calc_end]
fig,ax = plt.subplots(figsize=(15,4), dpi=100, facecolor="w")
caxes = ax.matshow(calc_rec, vmin = 0, vmax = 1, cmap = "Greys", aspect = "auto")
fig.colorbar(caxes)
plt.xticks(np.arange(0, calc_end - calc_start, 100), np.arange(calc_start, calc_end, 100))
plt.ylabel("\u0394 F / F", fontsize = 14)
plt.xlabel("Time", fontsize = 14)
plt.savefig(res_path + "neurons/calc_records_" + name + "_select_data_matrix.png", bbox_inches='tight')
plt.savefig(res_path + "neurons/calc_records_" + name + "_select_data_matrix.pdf", bbox_inches='tight')
plt.show()


In [ ]:
'''
supp neurons
'''

name = "supp"
ele_start, ele_end = calc_start / 10 + 10.4, calc_end / 10 + 10.4
ele_start_row, ele_end_row = int((ele_start - ele_start_second) * 1000), int((ele_end - ele_start_second) * 1000)
print("ele_start_row, ele_end_row:", ele_start_row, ele_end_row)

# plot ecog data
ele_rec = ele_fp26[ele_start_row : ele_end_row]
fig,ax = plt.subplots(figsize=(11,4), dpi=100, facecolor="w")
plt.plot(ele_rec, color = "#17202A", linewidth = 0.4, alpha = 0.8)
plt.title("ECoG signal", fontsize = 18)
plt.ylabel("(mV)", fontsize = 14)
plt.xlabel("Time", fontsize = 14)
plt.xticks(np.arange(0, ele_end_row - ele_start_row , 10000), np.arange(calc_start, calc_end , 100))
plt.show()

# load neuron ids
select_data_ids = np.load(res_path + "neurons/neurons_ids_" + name + "_all_start.npy")  ## change the file name
print("select_data_ids:", len(select_data_ids))

# plot calcium data 1
fig,ax = plt.subplots(figsize=(11,4), dpi=100, facecolor="w")
for i in select_data_ids:
    plt.plot(denoised_data[i, calc_start : calc_end], alpha = 0.8, color = "#633974", linewidth = 0.8)
plt.title("supp neurons", fontsize = 18)
plt.xticks(np.arange(0, calc_end - calc_start, 100), np.arange(calc_start, calc_end, 100))
plt.ylabel("\u0394 F / F", fontsize = 14)
plt.xlabel("Time", fontsize = 14)
plt.savefig(res_path + "neurons/calc_records_" + name + "_select_data_line.png", bbox_inches='tight')
plt.savefig(res_path + "neurons/calc_records_" + name + "_select_data_line.pdf", bbox_inches='tight')
plt.show()

# plot calcium data 2
calc_rec = denoised_data[select_data_ids, calc_start : calc_end]
fig,ax = plt.subplots(figsize=(15,4), dpi=100, facecolor="w")
caxes = ax.matshow(calc_rec, vmin = 0, vmax = 1, cmap = "Greys", aspect = "auto")
fig.colorbar(caxes)
plt.xticks(np.arange(0, calc_end - calc_start, 100), np.arange(calc_start, calc_end, 100))
plt.ylabel("\u0394 F / F", fontsize = 14)
plt.xlabel("Time", fontsize = 14)
plt.savefig(res_path + "neurons/calc_records_" + name + "_select_data_matrix.png", bbox_inches='tight')
plt.savefig(res_path + "neurons/calc_records_" + name + "_select_data_matrix.pdf", bbox_inches='tight')
plt.show()
